In [3]:
!pip install wandb
!pip install swig
!pip install gym[all]
!pip install pygame

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 28.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.0/297.0 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.9 MB/s eta 0:00

In [ ]:
import gym
import numpy as np
import torch
import random
import math
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

# Define a simple policy network
class PolicyNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, gamma=0.99, lr=0.001):
        super(PolicyNetwork, self).__init__()
        self.lr = lr
        self.gamma = gamma
        self.l1 = nn.Linear(input_dim, hidden_dim)
        self.l2 = nn.Linear(hidden_dim, output_dim)
        self.optimizer = optim.Adam(self.parameters(), lr=self.lr)

    def forward(self, x):
        x = torch.relu(self.l1(x))
        x = self.l2(x)
        return torch.softmax(x, dim=-1)

# Function to collect trajectories
def collect_trajectories(policy_network, env):
    trajectories = []
    total_reward = 0
    timesteps = 0
    episode = {'states': [], 'actions': [], 'rewards': []}
    state = env.reset()
    done = False
    while not done:
        timesteps += 1
        state = torch.tensor(state, dtype=torch.float32)
        action_probs = policy_network(state)
        action_dist = Categorical(action_probs)
        action = action_dist.sample()
        next_state, reward, done, _ = env.step(action.item())
        total_reward += reward
        episode['states'].append(state)
        episode['actions'].append(action)
        episode['rewards'].append(reward)
        state = next_state

    trajectories.append(episode)
    return trajectories, total_reward, timesteps

# Function to compute discounted rewards
def compute_discounted_rewards(rewards, gamma):
    discounted_rewards = []
    running_add = 0
    for r in reversed(rewards):
        running_add = running_add * gamma + r
        discounted_rewards.insert(0, running_add)
    return discounted_rewards

def smoothing(array):
  averages = []
  for i in range(0, len(array), 5):
      chunk = array[i:i+5]
      avg = sum(chunk) / len(chunk)
      averages.append(avg)
  return averages

# Function to update the policy network with entropy regularization
def update(policy_network, trajectories, gamma, entropy_weight):
    for episode in trajectories:
        loss = 0
        entropy = 0
        discounted_rewards = compute_discounted_rewards(episode['rewards'], gamma)
        baseline = np.mean(discounted_rewards)
        for state, action, dr in zip(episode['states'], episode['actions'], discounted_rewards):
            action_probs = policy_network(state)
            action_dist = Categorical(action_probs)
            log_prob = action_dist.log_prob(action)
            loss -= log_prob * dr
        policy_network.optimizer.zero_grad()
        loss.backward()
        policy_network.optimizer.step()

# Main function for training
def train():
    # Set up environment
    random.seed(350)
    env = gym.make('LunarLander-v2')
    input_dim = env.observation_space.shape[0]
    output_dim = env.action_space.n
    hidden_dim = 256

    # Hyperparameters
    n_episodes = 1000
    gamma = 0.99
    entropy_weight = 0.01  # Adjust this parameter to control the strength of entropy regularization

    policy_network = PolicyNetwork(input_dim, hidden_dim, output_dim, lr=0.001)
    rewards_array = []
    timesteps_array = []

    # Training loop
    for episode in range(n_episodes):
        trajectories, rewards, timesteps = collect_trajectories(policy_network, env)
        update(policy_network, trajectories, gamma, entropy_weight)
        rewards_array.append(rewards)
        timesteps_array.append(timesteps)
        print("Episode {}: Total Reward: {:.2f} Timesteps: {}".format(episode + 1, rewards, timesteps))

    smoothed_R1 = smoothing(rewards_array)
    smoothed_S1 = smoothing(timesteps_array)

    policy_network_2 = PolicyNetwork(input_dim, hidden_dim, output_dim, lr=0.01)
    rewards_array2 = []
    timesteps_array2 = []

    # Training loop
    for episode in range(n_episodes):
        trajectories, rewards, timesteps = collect_trajectories(policy_network_2, env)
        update(policy_network_2, trajectories, gamma, entropy_weight)
        rewards_array2.append(rewards)
        timesteps_array2.append(timesteps)
        print("Episode {}: Total Reward: {:.2f} Timesteps: {}".format(episode + 1, rewards, timesteps))


    smoothed_R2 = smoothing(rewards_array2)
    smoothed_S2 = smoothing(timesteps_array2)

    policy_network_3 = PolicyNetwork(input_dim, hidden_dim, output_dim, lr=0.1)
    rewards_array3 = []
    timesteps_array3 = []


    # Training loop
    for episode in range(n_episodes):
        trajectories, rewards, timesteps = collect_trajectories(policy_network_3, env)
        update(policy_network_3, trajectories, gamma, entropy_weight)
        rewards_array3.append(rewards)
        timesteps_array3.append(timesteps)
        print("Episode {}: Total Reward: {:.2f} Timesteps: {}".format(episode + 1, rewards, timesteps))

    smoothed_R3 = smoothing(rewards_array3)
    smoothed_S3 = smoothing(timesteps_array3)



    # Plotting
    plt.title("Performance of REINFORCE (in rewards) with Entropy Regularization in Lunar Lander")
    plt.plot(range(1, 201), smoothed_R1, label="lr=0.001")
    plt.plot(range(1, 201), smoothed_R2, label="lr=0.01")
    plt.plot(range(1, 201), smoothed_R3, label="lr=0.1")
    plt.xlabel("Episodes")
    plt.ylabel("Total Rewards")
    plt.legend()
    plt.show()

    plt.title("Performance of REINFORCE (in steps) with Entropy Regularization in Lunar Lander")
    plt.plot(range(1, 201), smoothed_S1, label="lr=0.001")
    plt.plot(range(1, 201), smoothed_S2, label="lr=0.01")
    plt.plot(range(1, 201), smoothed_S3, label="lr=0.1")
    plt.xlabel("Episodes")
    plt.ylabel("Timesteps")
    plt.legend()
    plt.show()

    env.close()

# Run the training
train()


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode 1: Total Reward: -114.16 Timesteps: 79
Episode 2: Total Reward: -127.27 Timesteps: 70
Episode 3: Total Reward: -202.29 Timesteps: 128
Episode 4: Total Reward: -355.37 Timesteps: 91
Episode 5: Total Reward: -389.58 Timesteps: 175
Episode 6: Total Reward: -125.08 Timesteps: 100
Episode 7: Total Reward: -90.84 Timesteps: 67
Episode 8: Total Reward: -431.99 Timesteps: 83
Episode 9: Total Reward: -157.06 Timesteps: 89
Episode 10: Total Reward: -93.95 Timesteps: 64
Episode 11: Total Reward: -205.96 Timesteps: 75
Episode 12: Total Reward: -394.42 Timesteps: 109
Episode 13: Total Reward: -311.58 Timesteps: 90
Episode 14: Total Reward: -233.61 Timesteps: 110
Episode 15: Total Reward: -130.22 Timesteps: 102
Episode 16: Total Reward: -139.49 Timesteps: 101
Episode 17: Total Reward: -185.55 Timesteps: 109
Episode 18: Total Reward: -159.66 Timesteps: 108
Episode 19: Total Reward: -318.97 Timesteps: 87
Episode 20: Total Reward: -186.01 Timesteps: 121
Episode 21: Total Reward: -150.28 Timeste